In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report

In [2]:
# Read in the data
df = pd.read_csv(Path('../data_csv/merged_air_dia.csv'))
# drop rows with null values
df = df.dropna()

# print out the dataframe
df

,city,pm25,pm10,population,color_pm10,color_pm25,uniquezip,state,data_value_type,data_value,low_confidence_limit,high_confidence_limit
0,Anchorage,5.650000,11.781883,380821,green,green,0203000-02020002501,Alaska,Crude prevalence,7.4,6.9,7.8
1,Tempe,7.350000,15.326874,161780,green,green,0473000-04013319907,Arizona,Crude prevalence,7.0,6.4,7.9
2,Yuma,8.300000,17.307898,195751,green,green,0485540,Arizona,Crude prevalence,11.1,11.0,11.3
3,Folsom,6.250000,13.033056,72199,green,green,0624638,California,Crude prevalence,7.1,7.0,7.3
4,San Francisco,7.500000,15.639667,870887,green,green,0667000,California,Crude prevalence,8.6,8.5,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...
12249,Charleston,7.300000,15.222609,227078,green,green,5414600-54039002000,West Virginia,Crude prevalence,10.3,9.3,11.3
12250,Yakima,8.550000,17.829220,243231,green,green,5380010-53077002802,Washington,Crude prevalence,9.0,8.4,9.5
12251,Yakima,8.550000,17.829220,243231,green,green,5380010-53077001000,Washington,Crude prevalence,8.9,8.4,9.4
12252,Tacoma,7.066667,14.736042,198397,green,green,5370000-53053061400,Washington,Crude prevalence,13.0,12.5,13.6


In [3]:
# create data frame for the columns that we do not need to scale or convert
unchanged_df = df[['pm25','pm10','population','data_value']]
unchanged_df.head()

,pm25,pm10,population,data_value
0,5.65,11.781883,380821,7.4
1,7.35,15.326874,161780,7.0
2,8.30,17.307898,195751,11.1
3,6.25,13.033056,72199,7.1
4,7.50,15.639667,870887,8.6


In [4]:
# check types for each column
df.dtypes

city                      object
pm25                     float64
pm10                     float64
population                 int64
color_pm10                object
color_pm25                object
uniquezip                 object
state                     object
data_value_type           object
data_value               float64
low_confidence_limit     float64
high_confidence_limit    float64
dtype: object

In [5]:
# drop high and low confidence limits for machine learning model so there is no bias
df.drop(columns=['low_confidence_limit','high_confidence_limit'],inplace=True)
df.head()

,city,pm25,pm10,population,color_pm10,color_pm25,uniquezip,state,data_value_type,data_value
0,Anchorage,5.65,11.781883,380821,green,green,0203000-02020002501,Alaska,Crude prevalence,7.4
1,Tempe,7.35,15.326874,161780,green,green,0473000-04013319907,Arizona,Crude prevalence,7.0
2,Yuma,8.30,17.307898,195751,green,green,0485540,Arizona,Crude prevalence,11.1
3,Folsom,6.25,13.033056,72199,green,green,0624638,California,Crude prevalence,7.1
4,San Francisco,7.50,15.639667,870887,green,green,0667000,California,Crude prevalence,8.6


In [6]:
# Convert columns that are objects to integers using onehotencoder
enc = OneHotEncoder(sparse=False)

# Fit and transform the onehotencoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[['color_pm25','color_pm10','data_value_type']]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(['color_pm25','color_pm10','data_value_type'])
encode_df.head()

,color_pm25_green,color_pm25_yellow,color_pm10_darkred,color_pm10_green,color_pm10_yellow,data_value_type_Age-adjusted prevalence,data_value_type_Crude prevalence
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [7]:
# Use pandas.get_dummies to convert columns with a large amount of unique entries

dummies_df = pd.get_dummies(df[['city','state']], columns=['city', 'state'])
dummies_df.head()

,city_Albany,city_Albuquerque,city_Allentown,city_Anaheim,city_Anchorage,city_Apple Valley,city_Atlanta,city_Auburn,city_Baltimore,city_Baton Rouge,...,state_Rhode Island,state_South Carolin,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Merge encoded dataframes with the remaining columns
ml_df = pd.concat([encode_df, unchanged_df, dummies_df],axis=1)
ml_df.head()

,color_pm25_green,color_pm25_yellow,color_pm10_darkred,color_pm10_green,color_pm10_yellow,data_value_type_Age-adjusted prevalence,data_value_type_Crude prevalence,pm25,pm10,population,...,state_Rhode Island,state_South Carolin,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,5.65,11.781883,380821,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.35,15.326874,161780,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,8.30,17.307898,195751,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,6.25,13.033056,72199,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.50,15.639667,870887,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# check to make sure all of the data types are correct for ML
ml_df.dtypes

color_pm25_green       float64
color_pm25_yellow      float64
color_pm10_darkred     float64
color_pm10_green       float64
color_pm10_yellow      float64
                        ...   
state_Utah               uint8
state_Vermont            uint8
state_Virginia           uint8
state_Washington         uint8
state_West Virginia      uint8
Length: 170, dtype: object

In [10]:
# Use standard scaler to help scale the data to train the model
scaler = StandardScaler()

# Want to scale the columns from data that are originally integers
cols_to_scale = ['pm10','pm25','population']
scaled_data = scaler.fit_transform(ml_df[cols_to_scale])
scaled_df = pd.DataFrame(scaled_data, columns=cols_to_scale)
scaled_df

,pm10,pm25,population
0,-1.280024,-1.280024,-0.823852
1,-0.396255,-0.396255,-0.899720
2,0.097616,0.097616,-0.887954
3,-0.968105,-0.968105,-0.930747
4,-0.318275,-0.318275,-0.654112
...,...,...,...
12249,-0.422248,-0.422248,-0.877103
12250,0.227582,0.227582,-0.871508
12251,0.227582,0.227582,-0.871508
12252,-0.543550,-0.543550,-0.887037


In [11]:
# drop the original columns that we scaled and add in the scaled columns to our ml_df
ml_df.drop(columns=cols_to_scale,inplace=True)
ml_df = pd.concat([ml_df, scaled_df],axis=1)
ml_df

,color_pm25_green,color_pm25_yellow,color_pm10_darkred,color_pm10_green,color_pm10_yellow,data_value_type_Age-adjusted prevalence,data_value_type_Crude prevalence,data_value,city_Albany,city_Albuquerque,...,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,pm10,pm25,population
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.4,0,0,...,0,0,0,0,0,0,0,-1.280024,-1.280024,-0.823852
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.0,0,0,...,0,0,0,0,0,0,0,-0.396255,-0.396255,-0.899720
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,11.1,0,0,...,0,0,0,0,0,0,0,0.097616,0.097616,-0.887954
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.1,0,0,...,0,0,0,0,0,0,0,-0.968105,-0.968105,-0.930747
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,8.6,0,0,...,0,0,0,0,0,0,0,-0.318275,-0.318275,-0.654112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12249,1.0,0.0,0.0,1.0,0.0,0.0,1.0,10.3,0,0,...,0,0,0,0,0,0,1,-0.422248,-0.422248,-0.877103
12250,1.0,0.0,0.0,1.0,0.0,0.0,1.0,9.0,0,0,...,0,0,0,0,0,1,0,0.227582,0.227582,-0.871508
12251,1.0,0.0,0.0,1.0,0.0,0.0,1.0,8.9,0,0,...,0,0,0,0,0,1,0,0.227582,0.227582,-0.871508
12252,1.0,0.0,0.0,1.0,0.0,0.0,1.0,13.0,0,0,...,0,0,0,0,0,1,0,-0.543550,-0.543550,-0.887037


In [12]:
# Split preprocessed data into features and target arrays
y = ml_df['data_value'].values
X = ml_df.drop(['data_value'],1).values

/Users/Kelly/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# check to see the shape of X
X.shape

(12254, 169)

In [14]:
y.shape # this is one column so that is what we want

(12254,)

In [15]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
# Create an object for linear regression
model = LinearRegression()

# time how long it takes to fit the model
start = datetime.now()

# Fit the linear regression model to the training set
model.fit(X_train,y_train)
end = datetime.now()

# print total time to fit and the RAM and CPU for machine
time_to_fit = end - start
print('Time to fit the model:',time_to_fit)
print('This code was run on a computer with memory: 4 GB 1600 MHz DDR3 and processor: 1.6 GHz Dual-Core Intel Core i5.')

Time to fit the model: 0:00:00.309873
This code was run on a computer with memory: 4 GB 1600 MHz DDR3 and processor: 1.6 GHz Dual-Core Intel Core i5.


In [17]:
# Predict the test set results
y_pred= model.predict(X_test)

In [18]:
# print summary stats to get r squared score
from statsmodels.api import OLS
OLS(y_test, X_test).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.334
Method:                 Least Squares   F-statistic:                     10.43
Date:                Sat, 28 May 2022   Prob (F-statistic):          1.72e-152
Time:                        09:39:29   Log-Likelihood:                -6468.5
No. Observations:                2451   AIC:                         1.320e+04
Df Residuals:                    2320   BIC:                         1.396e+04
Df Model:                         130                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             4.6423      0.309     15.016      0.000       4.036       5.249
x2             1.7616      0.514      3.428      0.001       0.754       2.769
x3             0.3348      0.613      0.546      0.585      -0.868       1.538
x4             3.1640      0.356      8.885      0.000       2.466       3.862
x5             2.9052      0.355      8.175      0.000       2.208       3.602
x6             3.4754      0.554      6.273      0.000       2.389       4.562
x7             2.9286      0.327      8.968      0.000       2.288       3.569
x8            -4.2087      2.986     -1.410      0.159     -10.064       1.647
x9            -0.0195      0.455     -0.043      0.966      -0.912       0.873
x10            3.6019      1.393      2.585      0.010       0.870       6.334
x11           -0.4319      1.134     -0.381      0.703      -2.656       1.792
x12           -1.2160      0.572     -2.125      0.034      -2.338      -0.094
x13            1.1497      3.511      0.327      0.743      -5.736       8.036
x14           -9.4324      2.847     -3.313      0.001     -15.016      -3.849
x15           -0.7588      1.773     -0.428      0.669      -4.235       2.718
x16            0.7804      0.288      2.709      0.007       0.215       1.345
x17            1.6322      0.505      3.231      0.001       0.641       2.623
x18           -3.2199      1.419     -2.269      0.023      -6.003      -0.437
x19           -0.6618      1.244     -0.532      0.595      -3.102       1.778
x20           -1.6761      1.997     -0.839      0.401      -5.592       2.239
x21           -1.9280      0.904     -2.132      0.033      -3.702      -0.154
x22            0.5110      2.454      0.208      0.835      -4.302       5.324
const       3.649e-15   4.63e-15      0.788      0.431   -5.43e-15    1.27e-14
x23            3.3350      1.318      2.531      0.011       0.751       5.919
x24           -0.8179      1.164     -0.703      0.482      -3.100       1.464
x25           -0.4274      0.953     -0.448      0.654      -2.297       1.442
x26           -2.2400      1.807     -1.239      0.215      -5.784       1.304
x27            0.2977      1.197      0.249      0.804      -2.050       2.646
x28            3.2091      0.353      9.102      0.000       2.518       3.901
x29           -0.2524      2.223     -0.114      0.910      -4.612       4.107
x30            1.1609      1.069      1.086      0.277      -0.934       3.256
x31           -1.0677      0.401     -2.663      0.008      -1.854      -0.281
x32            0.1033      1.383      0.075      0.940      -2.608       2.815
x33           -1.9904      2.942     -0.676      0.499      -7.761       3.780
x34           -1.3658      0.673     -2.030      0.042      -2.685      -0.047
x35            1.8341      1.003      1.828      0.068      -0.134       3.802
x3

In [19]:
# get more summary stats to see mean squared error
import sklearn.metrics as metrics
def regression_results(y_test, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_test, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_test, y_pred) 
    mse=metrics.mean_squared_error(y_test, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_test, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_test, y_pred)
    r2=metrics.r2_score(y_test, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

regression_results(y_test,y_pred)

explained_variance:  0.3334
mean_squared_log_error:  0.0984
r2:  0.3334
MAE:  2.6526
MSE:  12.1231
RMSE:  3.4818
